In [1]:
# 爬虫简介
# 数据是机器学习的燃料，爬虫就是获取数据的挖掘机。

In [2]:
# 第一节 直接使用封装好的接口获取数据

In [3]:
# tushare获取股票数据
import pandas as pd
import tushare as ts

data = ts.get_hist_data('510050',start='2018-01-01',end='2018-01-05')
data.index = pd.to_datetime(data.index)
data = data[::-1]
data.head()

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
date,,,,,,,,,,,,,
2018-01-02,2.86,2.91,2.91,2.86,4776970.00,0.05,1.75,2.870,2.875,2.868,5108968.20,4969266.33,5051746.58
2018-01-03,2.91,2.95,2.91,2.91,5708743.00,0.00,0.00,2.874,2.879,2.868,5265180.70,5021250.08,4890998.78
2018-01-04,2.91,2.93,2.92,2.91,3084130.75,0.01,0.34,2.892,2.883,2.871,4165531.45,4922446.00,4812177.54
2018-01-05,2.92,2.94,2.93,2.92,3220088.25,0.01,0.34,2.906,2.887,2.875,3906197.70,4670167.18,4674675.88


In [4]:
# 第二节 requests库

In [5]:
# 官网：http://cn.python-requests.org/zh_CN/latest/
# 一句话简介：Requests 唯一的一个非转基因的 Python HTTP 库，人类可以安全享用。
# 不服的话请自行挑战urllib库。
# 安装：pip install requests

In [6]:
# 常用HTTP请求的功能
# Get: 请求指定的资源
# Head: 检查资源是否存在
# Put: 更新资源 
# Delete: 删除资源
# Post: 上传资源和表单

In [7]:
# HTTP请求结构：
# https://www.xxx.com/yyy?v1=a&v2=b
# 协议名称是https
# url是www.xxx.com/yyy
# 有2个参数v1和v2，对应值是a和b。

In [8]:
# http://httpbin.org/是个很好的测试和学习对象

In [9]:
# Get请求
import requests
r = requests.get("http://httpbin.org/get")
print(r.status_code) # 返回HTTP状态码
print(r.encoding) # 响应内容的编码
print(r.headers) # 响应的头部信息
print(r.text)
jdata = r.json()
print(type(jdata))
print(jdata)

200
None
{'Connection': 'keep-alive', 'Server': 'meinheld/0.6.1', 'Date': 'Sun, 28 Jan 2018 07:47:46 GMT', 'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'X-Powered-By': 'Flask', 'X-Processed-Time': '0.00121903419495', 'Content-Length': '268', 'Via': '1.1 vegur'}
{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.12.4"
  }, 
  "origin": "183.193.155.125", 
  "url": "http://httpbin.org/get"
}

<class 'dict'>
{'args': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Connection': 'close', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.12.4'}, 'origin': '183.193.155.125', 'url': 'http://httpbin.org/get'}


In [10]:
# 带参数的get请求
payload = {'key1': 'value1', 'key2': 'value2'}
r = requests.get("http://httpbin.org/get", params=payload)
print(r.json()['args'])

{'key1': 'value1', 'key2': 'value2'}


In [11]:
# Post请求
payload = {'key1': 'value1', 'key2': 'value2'}
r = requests.post("http://httpbin.org/post", data=payload)
print(r.json()['form'])

{'key1': 'value1', 'key2': 'value2'}


In [12]:
# Cookie处理
r = requests.get('http://www.douban.com')
print(r.headers)
for k in r.cookies.keys():
    print(k, r.cookies[k])

{'Date': 'Sun, 28 Jan 2018 07:48:06 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=30', 'Vary': 'Accept-Encoding', 'X-Xss-Protection': '1; mode=block', 'X-Douban-Mobileapp': '0', 'Expires': 'Sun, 1 Jan 2006 01:00:00 GMT', 'Pragma': 'no-cache', 'Cache-Control': 'must-revalidate, no-cache, private', 'Set-Cookie': 'll="108296"; path=/; domain=.douban.com; expires=Mon, 28-Jan-2019 07:48:06 GMT, bid=K9odnGsB_DE; Expires=Mon, 28-Jan-19 07:48:06 GMT; Domain=.douban.com; Path=/', 'X-DOUBAN-NEWBID': 'K9odnGsB_DE', 'X-DAE-Node': 'sindar13d', 'X-DAE-App': 'sns', 'Server': 'dae', 'Strict-Transport-Security': 'max-age=15552000;', 'Content-Encoding': 'gzip'}
bid K9odnGsB_DE
ll "108296"


In [13]:
# 二进制内容处理
from PIL import Image
from io import BytesIO

r = requests.get('http://image1.ljcdn.com/shfdfs-image/m_fit,w_960,q_90/20180127/81244784-d394-47f4-8a2c-77826b8f9e0f.jpg')
image = Image.open(BytesIO(r.content))
image.save('lianjia.jpg')

In [14]:
# 使用requests库从新浪获取股票数据
def get_stock_info(code):
    api = 'http://hq.sinajs.cn/list='
    r = requests.get(api + code)
    data_str = r.text.split('=')[1].replace('"', '')[:-2]
    infos = data_str.split(',')
    s = infos[0] + ': '
    s += infos[1] + ', ' # 开
    s += infos[3] + ', ' # 收
    s += infos[4] + ', ' # 高
    s += infos[5] + ', ' # 低
    print(s)
    
get_stock_info('sh601006')

大秦铁路: 10.010, 10.220, 10.240, 9.970, 


In [15]:
# 第3节 css定位器与bs4网页解析

In [16]:
# HTML基础知识
# <标签 属性_1=值_1 属性_2=值_2>内容...</标签>
# id和class作为2个特殊标签被特别对待

In [17]:
# css选择器（可以使用浏览器的开发者工具检查或生成选择器）
# 选择标签：直接输入标签名字，如head/ul，用 > 表示层级关系，比如ul > li（当中的空格必须保留）。
# 选择id：#id，比如百度搜索的“百度一下”按钮可以用 #su 定位。
# 选择class: .classname，比如百度搜索的搜索框可以用 .s_ipt 定位。
# id和class的区别：理论上id是唯一的。而一个标签则可以同时属于多个class，可以用 .class_1.class_2 表示选择该标签的2个class进行匹配。
# 元素属性选择：[属性=值]，多个属性的话直接连接即可，[属性_1=值_1][属性_1=值_2]
# 例子，选择以下元素<div id=su class=names size=100 show=popup>Some Text</div>
# - div#su.names[size=100][show=popup]
# - 因为id一般唯一，所以 #su 也足够表达。

In [18]:
# bs4网页解析
# 安装：pip install bs4

In [19]:
# 结合requests找到百度提交按钮
import requests
from bs4 import BeautifulSoup

resp = requests.get('http://www.baidu.com')
text = resp.text.encode('iso-8859-1').decode('utf-8') # 编码问题校正
bs = BeautifulSoup(text, 'html5lib')
su = bs.select('#su')[0]
su.get('value')

'百度一下'

In [20]:
# 抓取全国各地邮政编码与长途区号
BASE_URL = 'http://www.ip138.com'
UA = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
HEADERS = {'user-agent':UA}

def handle_zip_code(province, url):
    print(province)
    resp = requests.get(url, headers=HEADERS)
    text = resp.text.encode('iso-8859-1').decode('gbk')
    bs = BeautifulSoup(text, 'html5lib')
    rows = bs.select('table.t12 > tbody > tr')
    for i in range(1, len(rows)):
        items = rows[i].select('td')
        t_1 = items[0].text.strip()
        if t_1:
            print('\t%s, %s' % (t_1, items[1].text.strip()))
        if len(items) > 4:
            t_2 = items[3].text.strip()
            if t_2:
                print('\t%s, %s' % (t_2, items[4].text.strip()))
    
resp = requests.get(BASE_URL + '/post', headers=HEADERS)
text = resp.text.encode('iso-8859-1').decode('gbk')
bs = BeautifulSoup(text, 'html5lib')
for item in bs.select('div#newAlexa > table.t4 > tbody > tr > td > a'):
    handle_zip_code(item.text, BASE_URL + item.get('href'))
    break # 不然显示内容太多

北京市
	通县, 101100
	东城区, 100010
	西城区, 100032
	崇文区, 100061
	宣武区, 100054
	朝阳区, 100020
	丰台区, 100071
	石景山区, 100043
	海淀区, 100080
	门头沟区, 102300
	房山区, 102400
	通州区, 101100
	顺义区, 101300
	昌平区, 102200
	大兴区, 102600
	怀柔区, 101400
	平谷区, 101200
	密云区, 101500
	延庆区, 102100
	北京市, 100000


In [21]:
# Selenium安装配置
# 1. 安装python的selenium包：pip install selenium
# 2. Selenium驱动（Chrome）下载：https://sites.google.com/a/chromium.org/chromedriver/downloads
# 3. 驱动下载后放在任意位置，但需要把路径添加到PATH环境变量。

In [22]:
# 使用示例
from selenium import webdriver
options = webdriver.ChromeOptions() # 创建driver的参数，可选。
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
# options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(chrome_options=options)
driver.get('http://www.baidu.com')
element = driver.find_element_by_css_selector('#su')
print(element.get_attribute('value'))
driver.quit()

百度一下


In [23]:
# 模拟百度搜索
import time

driver = webdriver.Chrome()
driver.get('http://www.baidu.com')
time.sleep(2)
search_input = driver.find_element_by_css_selector('input#kw')
search_input.send_keys('python')
time.sleep(2)
submit = driver.find_element_by_css_selector('input#su')
submit.click() # 如果click无效，可以尝试driver.execute_script('arguments[0].click();', element)
time.sleep(2)
results = driver.find_elements_by_css_selector('h3 > a')
for result in results:
    print(result.text, result.get_attribute('href'))
driver.quit()

python首页--从入门到精通python http://www.baidu.com/baidu.php?url=Ks0000amqZra9B-65khmGxlB30cAvgOtNZfeGEg2xoXpsRyPpNb_Rv91ol4cRM-p1Qj049H7emSKPq5qkLj1Z90inQpEhdLcd_v7bIdctZsIRQGZ2jpZLmajvZE_yoXf6ZjcEz1p1k1SUtwkX9rQeuq9VMFiKUOBmshTlUyYHx9iGivx9f.7b_jxgC9xfwKfCsmqDZxxgMLvmtqI-hHIk133N2s1f_TXMGlXkf.U1Yk0ZDqTZ-YpAq80ZKGm1Yk0ZfqTZ-YpAq80A-V5HczPfKM5yF-TZns0ZNG5yF9pywd0ZKGujYz0APGujYYnjR0UgfqnH0krNtknjDLg1nvnjD0pvbqn0KzIjYdPjn0uy-b5HDYPWPxnWDsrjKxnHTsnj7xnW04nW60mhbqnW0Y0AdW5HTkP16sPWmkn7tLnH61nWcYnWKxn0KkTA-b5H00TyPGujYs0ZFMIA7M5H00mycqn7ts0ANzu1Ys0ZKs5H00UMus5H08nj0snj0snj00Ugws5H00uAwETjYs0ZFJ5H00uANv5gKW0AuY5H00TA6qn0KET1Ys0AFL5HDs0A4Y5H00TLCq0ZwdT1YkPWfknH6YPHfsnHf1P1bdPWmvPsKzug7Y5HDdnHTknWRLPWcvPHD0Tv-b5y7BnyRzPjIBnj0snj-9rHn0mLPV5HKDwRmdwD7DnHmdwRFawWm0mynqnfKsUWYs0Z7VIjYs0Z7VT1Ys0ZGY5H00UyPxuMFEUHYsg1Kxn0Kbmy4dmhNxTAk9Uh-bT1Ysg1Kxn0Ksmgwxuhk9u1Ys0AwWpyfqn0K-IA-b5iYk0A71TAPW5H00IgKGUhPW5H00Tydh5H00uhPdIjYs0AulpjYs0Au9IjYs0ZGsUZN15H00mywhUA7M5HD0UAuW5H00mLFW5HnYP10k&ck=0.0.0.0.0.0.0.0&shh=www

In [24]:
# 利用selenium制作截图工具
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 如果截图不正确请先检查option
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(chrome_options=options) # 带options的初始化
driver.get('http://news.sina.com.cn')
width = driver.execute_script('return Math.max(document.body.scrollWidth, document.body.offsetWidth, document.documentElement.clientWidth, document.documentElement.scrollWidth, document.documentElement.offsetWidth);')
height = driver.execute_script('return Math.max(document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight);')
print(width, height)
driver.set_window_size(width, height)
driver.save_screenshot('sina_news.jpg')
driver.quit()

1020 13817


In [25]:
# 作业
# 1. 利用www.okex.com提供的API抓取所有山寨币对比特币的报价，要求速度尽可能快。
# 2. 利用selenium实现豆瓣用户名/密码登陆，
#    如果有验证码，提示用户在命令行交互界面输入，然后再由selenium提交验证码。